In [1]:
from dmrApp import create_app, db
from dmrApp.models import Dmrs, Employees, Employeeroles, Post, Restaurants, Shifts, User, Standardwages
import numpy as np
from datetime import date, datetime
from datetime import timedelta
import datetime
from sqlalchemy import func
import pandas as pd
import os
proxyApp=create_app()
ctx=proxyApp.app_context()
ctx.push()

ModuleNotFoundError: No module named 'dmrApp'

In [2]:
dateFrom='2020-12-27'
dateTo='2021-01-30'
resId=3

In [6]:
shiftDf=pd.read_sql_table('shifts',db.engine)
employeerolesDf=pd.read_sql_table('employeeroles',db.engine)
restaurantsDf=pd.read_sql_table('restaurants',db.engine)
print('dateFrom::::',dateFrom,type(dateFrom))

#Build dictionary of weeks with Start and end dates
#get date ranges for tables
weekStartDt= datetime.datetime.strptime(dateFrom,'%Y-%m-%d').date()
weekEndDt=datetime.datetime.strptime(dateFrom,'%Y-%m-%d').date() + timedelta(days=6)
weeksDict={}
i=1
while weekEndDt< datetime.datetime.strptime(dateTo,'%Y-%m-%d').date():
    weeksDict[f'week{i}']=[weekStartDt,weekEndDt]
    weekStartDt=weekEndDt + timedelta(days=1)
    weekEndDt = weekEndDt + timedelta(days=7)
    i+=1
# final week calculation
weeksDict[f'week{i}']=[weeksDict[f'week{i-1}'][0] + timedelta(days=7),weeksDict[f'week{i-1}'][1] + timedelta(days=7)]

#build dataframe for one week's payroll if it exists
dfList=[]
# i=1
for x,y in weeksDict.items():
    df=shiftDf[(shiftDf['shiftDate']>=str(y[0])) & (shiftDf['shiftDate']<=str(y[1])) & (shiftDf['restaurantId']==resId)]
    if len(df)>0:
        df=df.groupby(['name','empId','employeeRolesId','restaurantId']).sum()
        df=df[['hoursWorked','shiftTips','shiftTipsShipgarten','wages']].copy()
        df.reset_index(inplace=True)
        df['Tips']=np.where((df['restaurantId'] >=3) & (df['restaurantId']<=6),df['shiftTipsShipgarten'],df['shiftTips'])
        df.set_index(['name','empId','employeeRolesId','restaurantId'], inplace=True)
        df1=df[['hoursWorked','Tips','wages']].copy()
        df1.columns=['Hours', 'Tips','Wages']
        dfList.append(df1)
    else:
        noShiftsColumn=f'No shifts {str(y[0])} thru {str(y[1])}'

if len(dfList)==0:
    pass
elif len(dfList)==1:
    df2=dfList[0]
else:
    df2=pd.merge(dfList[0],dfList[1],how='outer', on=['name','empId','employeeRolesId','restaurantId'], 
         suffixes=(' Week1',' Week2')).fillna(0)
    df2['Total Hours']=df2['Hours Week1']+df2['Hours Week2']
    df2['Total Tips']=df2['Tips Week1']+df2['Tips Week2']
    df2['Total Wages']=df2['Wages Week1']+df2['Wages Week2']

#get hourly rate from employee roles
rolesDf=employeerolesDf[['id','role','wage']].copy()
rolesDf.columns=['roleId','Role','Wage']
df2.reset_index(inplace=True)
df2.set_index('employeeRolesId')
df3=pd.merge(df2,rolesDf,how='left',left_on='employeeRolesId',right_on='roleId')

restaurantsDf.columns=['restaurantId','Restaurant Name']
df4=pd.merge(df3,restaurantsDf,how='left',left_on='restaurantId',right_on='restaurantId')


if len(df4.columns)==11:
    colNames=['Name','Restaurant Name','Role','Hours','Wages','Tips']
    payrollReportDf=df4[['name','Restaurant Name','Role','Hours','Wages','Tips']].copy()
    payrollReportDf['Week Start']=weeksDict['week1'][0]
    payrollReportDf['Shift Status']=noShiftsColumn
else:
    colNames=['Name','Restaurant Name','Role','Hours Week1','Hours Week2','Total Hours','Wages Week1',
                       'Wages Week2','Total Wages','Tips Week1','Tips Week2','Total Tips']
    payrollReportDf=df4[['name','Restaurant Name','Role','Hours Week1','Hours Week2','Total Hours','Wages Week1',
                       'Wages Week2','Total Wages','Tips Week1','Tips Week2','Total Tips']].copy()
    payrollReportDf['Week1 Start']=weeksDict['week1'][0]
    payrollReportDf['Week2 Start']=weeksDict['week2'][0]
    payrollReportDf['Total Tips']=payrollReportDf['Total Tips'].apply(lambda x: round(x, 2))
payrollReportDf.rename(columns={'name':'Name'},inplace=True)

dateFrom:::: 2020-12-27 <class 'str'>


In [13]:
payrollReportDf.iloc[:,3:]

Hours Week1  Hours Week2  Total Hours  Wages Week1  Wages Week2  \
0          1.0     1.000000     2.000000         2.13       2.1300   
1          1.0     0.000000     1.000000         2.13       0.0000   
2          0.0     2.683333     2.683333         0.00       5.7155   

   Total Wages  Tips Week1  Tips Week2  Total Tips Week1 Start Week2 Start  
0       4.2600       0.665    1.000000        1.67  2020-12-27  2021-01-03  
1       2.1300       0.665    0.000000        0.67  2020-12-27  2021-01-03  
2       5.7155       0.000    0.944282        0.94  2020-12-27  2021-01-03

In [17]:
for i in payrollReportDf.iloc[:,3:-2]:
    payrollReportDf[i]=payrollReportDf[i].apply(lambda x: round(x, 2))

In [18]:
payrollReportDf

Name    Restaurant Name       Role  Hours Week1  Hours Week2  \
0    Nick  Tysons Biergarten  bartender          1.0         1.00   
1    Zane  Tysons Biergarten  bartender          1.0         0.00   
2  Shayan  Tysons Biergarten  bartender          0.0         2.68   

   Total Hours  Wages Week1  Wages Week2  Total Wages  Tips Week1  Tips Week2  \
0         2.00         2.13         2.13         4.26        0.67        1.00   
1         1.00         2.13         0.00         2.13        0.67        0.00   
2         2.68         0.00         5.72         5.72        0.00        0.94   

   Total Tips Week1 Start Week2 Start  
0        1.67  2020-12-27  2021-01-03  
1        0.67  2020-12-27  2021-01-03  
2        0.94  2020-12-27  2021-01-03

In [39]:
import glob
from flask import current_app
filesForDelete=glob.glob(os.path.join(current_app.root_path, 'static/reports/*.xlsx'))
# filesForDelete=os.path.join(current_app.root_path, 'static/reports/')
# print(type(filesForDelete))
# print(filesForDelete)
# for file in filesForDelete:
#     print(file)
for file in filesForDelete:
    os.remove(file)

In [ ]:
file_path = '/tmp/file.txt'

try:
    os.remove(file_path)

In [57]:
dmrsDf=pd.read_sql_table('dmrs', db.engine)
# type(dmrsDf.iloc[1,1])
# dmrsDf.dmrDate=dmrsDf.dmrDate- pd.datetime(1899, 12, 31)
dmrsDf.head()

id    dmrDate  startCash  payout  sales  compSales  voidSales  cashDrop  \
0   1 2021-01-09        0.0     0.0    0.0        0.0        0.0       0.0   
1   2 2021-01-09        0.0     0.0    0.0        0.0        0.0       0.0   
2   3 2021-01-09        0.0     0.0    0.0        0.0        0.0       0.0   
3   4 2021-01-09        0.0     0.0    0.0        0.0        0.0       0.0   
4   6 2021-01-09        0.0     0.0    0.0        0.0        0.0       0.0   

   expectedCash  numHoursWorked  ...  creditCardTip  cashTip  tip  wages  \
0           0.0             0.0  ...            0.0      0.0  0.0   0.00   
1           0.0             2.0  ...            1.0      0.0  1.0   4.26   
2           0.0             2.0  ...            1.0      0.0  1.0   4.26   
3           0.0             1.0  ...            0.0      0.0  0.0   0.00   
4           0.0             1.0  ...            1.0      0.0  1.0   0.00   

                dmrTimeStamp  infoFrom infoFromCount restaurantId  \
0 2021-01-12 08:44:26.607924      None           NaN            3   
1 2021-01-23 15:42:16.746573      [63]           1.0            1   
2 2021-01-23 15:42:17.510930      [63]           1.0            2   
3 2021-01-12 08:45:03.167659         2           1.0            3   
4 2021-01-12 08:48:56.641502         5           1.0            3   

   restaurantGroupId  user_id  
0                2.0        1  
1                NaN        1  
2                NaN        1  
3                2.0        1  
4                2.0        1  

[5 rows x 22 columns]

In [47]:
dmrsDf.iloc[1,1]

Timedelta('44204 days 00:00:00')

In [60]:
timeStamp = datetime.datetime.now().strftime("%y%m%d_%H%M%S")
reportName=f'testDate_{timeStamp}.xlsx'
excelObj=pd.ExcelWriter(reportName, date_format='yyyy/mm/dd', datetime_format='yyyy/mm/dd')
workbook=excelObj.book
dmrsDf.to_excel(excelObj,sheet_name='dmrs', index=False)
dmrDateFormat = workbook.add_format({'num_format': 'yyyy/mm/dd'})
worksheet=excelObj.sheets['dmrs']
worksheet.set_column(1,1, 15, dmrDateFormat)
excelObj.close()